In [ ]:
import sys
sys.path.append('..')

# Helper imports
from data import CIFAR10, IMAGENETTE
import utils

# Numerical computing and display imports
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

# Other imports
from datetime import datetime
import logging
import pickle
import os

In [ ]:
# Load and Prepare the Data
folder_path = '../../images/'
dataset = 'imagenette'    
if dataset == 'cifar10':
    images = CIFAR10()
if dataset == 'imagenette':
    images = IMAGENETTE()
x = images.load()
images.save_to_disk(x, folder_path, num_images=110)
folder_path = utils.move_data_to_temp_ram(folder_path, ram_size_mb=50)

In [ ]:
hamm_distance = []

for j in range(5):
    target_path = f'../../images/{105+(j+1)}.bmp' 
    target_hash = utils.compute_hash(target_path)

    for i in range(100):
        img_path = f'../../images/{i+1}.bmp' 
        img_hash = utils.compute_hash(img_path)
        # Compute the hamming distance between the images
        hamm = utils.distance(img_hash, target_hash, 'hamming')
        hamm_distance.append(hamm/128)
        print(f'Relative Hamming Distance {(hamm/128)*100:.2f}%')

print(f'Average Relative Hamming Distance: {sum(hamm_distance)/len(hamm_distance) :.2f}')

In [ ]:
plt.bar(np.linspace(1, len(hamm_distance), num=len(hamm_distance)), hamm_distance)
plt.axhline(0.4, color='r', linestyle='--')
plt.ylabel('Hamming Distance')
plt.xlabel('Image Index')
#plt.title('PDQ Hamming Distance between 2 Distinct Images')